In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf 
import os
import pathlib
from scipy import signal
from scipy.io import wavfile 

model_path = '/content/tflite_audio_27mar.tflite'
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

Input Shape: [  1 129 170   1]
Input Type: <class 'numpy.float32'>
Output Shape: [1 4]
Output Type: <class 'numpy.float32'>


In [15]:
rate, samples = wavfile.read ('/content/Recording_10.wav')

#just remove this and provide audio of 1 sec, which is of 22050 samples
zero_padding = tf.zeros([22050] - tf.shape(samples), dtype=tf.float32)
waveform = tf.cast(samples, tf.float32)
equal_length = tf.concat([waveform, zero_padding], 0)
#remove above part

#this will work in rpi
f, t, spec = signal.stft(equal_length.numpy(), fs=22050, nperseg=255, noverlap = 124, nfft=256)
spectrogram = np.abs(spec)

#heavy printing to have some idea of things
print (spectrogram.dtype)
print (spectrogram.shape)

spec2 = np.reshape(spectrogram,(1, spectrogram.shape[0], spectrogram.shape[1], 1))
print (spec2.shape)

interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], spec2)
interpreter.invoke()

predictionx = interpreter.get_tensor(output_details[0]['index'])

output = np.argmax(predictionx[0])

#heavy conditions of programming
if output==0:
  print ('on')
elif output==1:
  print ('off')
elif output==2:
  print ('green')
elif output==3:
  print ('red')

float32
(129, 170)
(1, 129, 170, 1)
off
